In [120]:
import geopandas as gpd
import pandas as pd
import numpy as num
from shapely.geometry import Point

In [121]:
raw_data = gpd.read_file("nv_vest_20.shp")
census_data = gpd.read_file("nv_cvap_2020_bg.shp")
points = raw_data.copy()
census_points = census_data.copy()

In [122]:
#points['geometry'] = points['geometry'].centroid
# geoid is separated to STATEFP, COUNTYFP, VTDST --> Integrate them for GEOID

points = points.assign(GEOID20 = points.STATEFP.astype(str) + '' + points.COUNTYFP.astype(str) + '' + points.VTDST.astype(str))
points = pd.DataFrame(points.drop(columns=['STATEFP', 'COUNTYFP', 'VTDST']))
points.rename(columns={'G20PREDBID': 'DemocraticVote', 'G20PRERTRU': 'RepublicanVote', 
                       'G20PRELJOR': 'LibertarianVote', 'G20PREIBLA': 'IndependentVote', 
                       'G20PREONON': 'OtherPartiesVote'}, inplace=True)

#raw_data.head()
points.head()
#gdf = gpd.GeoDataFrame(points[['COUNTY', 'NAME', 'DemocraticVote', 'RepublicanVote','LibertarianVote', 'IndependentVote', 'OtherPartiesVote', 'geometry']],)
#gdf.to_file('D:/Preprocessing/nevada_vote.json', driver='GeoJSON')

,COUNTY,NAME,DemocraticVote,RepublicanVote,LibertarianVote,IndependentVote,OtherPartiesVote,geometry,GEOID20
0,Churchill,CHURCHILL PRECINCT 14,100,500,12,1,8,"POLYGON Z ((-119.06227 39.34643 0.00000, -119....",32001000014
1,Churchill,CHURCHILL PRECINCT 19,117,504,13,1,6,"POLYGON Z ((-119.11763 39.50918 0.00000, -119....",32001000019
2,Esmeralda,ESMERALDA GOLDFIELD PRECINCT 1,36,179,4,1,3,"POLYGON Z ((-117.43051 37.78696 0.00000, -117....",32009000001
3,Esmeralda,ESMERALDA SILVER PEAK PRECINCT 2,4,42,0,0,1,"POLYGON Z ((-117.87888 37.91707 0.00000, -117....",32009000002
4,Esmeralda,ESMERALDA FISH LAKE VALLEY PRECINCT 3,30,165,2,0,2,"POLYGON Z ((-118.42300 37.89637 0.00000, -118....",32009000003


In [123]:
# NHS = Not Hispanic or Latino
# AIA = American Indian or Alaska Native
# ASN = Asian
# BLK = African American
# NHP = Native Hawaiian or Other Pacific Islander Alone


census_points.rename(columns={'C_TOT20': 'Total', 'C_NHS20':'NHS', 'C_AIA20':'AIA',  
                'C_ASN20':'Asian', 'C_BLK20':'Black', 'C_NHP20':'NHP', 
                'C_WHT20':'White', 'C_HSP20':'HispanicOrLatino' }, inplace=True)
census_points = census_points.filter(items=['GEOID20', 'AIA', 'Asian', 'Black',
                                            'White', 'HispanicOrLatino','geometry'])
census_points.head()

#census_points.to_file('D:/Preprocessing/nevada_census.json', driver='GeoJSON')

,GEOID20,AIA,Asian,Black,White,HispanicOrLatino,geometry
0,320030032492,0,105,105,1320,360,"POLYGON ((-115.34261 36.18842, -115.34241 36.1..."
1,320030032491,50,300,35,1195,110,"POLYGON ((-115.33553 36.19193, -115.33391 36.1..."
2,320030032512,4,300,10,820,170,"POLYGON ((-115.32364 36.19362, -115.32363 36.1..."
3,320030032502,35,425,185,2125,430,"POLYGON ((-115.34069 36.18167, -115.34006 36.1..."
4,320030032482,0,105,670,310,450,"POLYGON ((-115.27882 36.21438, -115.27881 36.2..."
